Author: Nathaniel Blair-Stahn<br>
Date: July 30, 2019<br>
Purpose: Verify bug fix for ylds in Nigeria output

In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
# from ipywidgets import interact, IntSlider

pd.set_option('display.max_rows', 8)

!date
!whoami

Tue Jul 30 18:28:16 PDT 2019
ndbs


In [2]:
import vivarium_conic_sqlns.verification_and_validation.sqlns_output_processing as sop

In [3]:
result_dir = '/share/costeffectiveness/results/sqlns/presentation/nigeria/2019_07_30_00_01_45'

In [ ]:
output = sop.clean_and_aggregate(result_dir, 'output.hdf')
output